## Current RoadMap
### Week 6: Advanced Pandas with data cleaning.
* Loading data and cleaning
* Filerting, grouping and plotting data

### Week 7: Pulling it all together 
* json through pandas through plotting through saving updated data

# 1. Load in data + check the data
Let's start by loading in and checking the cleanliness of dataset from Hestia...

In [ ]:
# Load in the libraries we require for today
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Let's use pandas to load in Excel data
If this does not work, use: !pip install openpyxl


In [ ]:
# Load Maize data from Hestia 
df = pd.read_excel("MaizePig.xlsx")  

In [ ]:
# Look at the data
df.head()

Hmm, the data looks a little messy (265 columns is quite a lot!) and the column names are not lined up correctly. Obviously, we may just go into Excel and fix this manually, but I will also show you how to clean this using Python directly. 

#### 🤨 TASK
We've looked at the header, and can see there is clearly problems. In the cell below look at the footer of the data to see if there is any other obvious issues.


Next, let's look at some summary statistics of the data, this will give us some indication about missing data and data types

In [ ]:
df.describe()
# df.info()

Clearly the column names are being put as a seperate row, this is a common issue with Excel sheets from different sources. Before working with this data we need to fix this

# 2. Cleaning data in python 
From first look (and some investigation in Excel), it looks like we need to combine the first two rows ~, and maybk if we need to change some of the data types, for example strings to integers.
 

Open in excel to view problems and get an idea of what cleaning needs to be done. First row has useful information.

First look at the columns

In [ ]:
df.columns.unique()

Hmm, let's look at the columns without these Unnamed columns.   ~
We can achieve this reduced view with list comprehension and ask the list not to include the term 'Unnamed'

In [ ]:
[col_name for col_name in list(df.columns.unique()) if 'Unnamed' not in col_name]

There's still a problem here, so let's get only the useful information from rows 1 and 2 but need to think of a way to combine them

In [ ]:
df.iloc[0].unique()

#### 🤨 TASK
Look at unique entries in row 2

We can tell that we need to make new formatted column names which are a combination of row 1 and 2, discarding the original column names (row 0)

 changing '-' to na/none

In [ ]:
dash = df.iloc[0][1]
dash

We can use the `.replace()` method to replace the dash with np.nan (which is float type). Doing this will later allow us to use the `.dropna()` method, which drops all NaN values. Remember NaN means Not a Number.

In [ ]:
df = df.replace(dash, np.nan) # using numpy NaN so it works with summary stats

In [ ]:
df.head()

Let's get the first two rows, as these look like they have our column names in

In [ ]:
row1 = df.iloc[0]
row2 = df.iloc[1]

Let's look at the first 5 unique values to check...

In [ ]:
row1.unique()[:5]

In [ ]:
row2.unique()[:5]

Next, we can create a combined list of new column names based on an assumption that if the column name is not in row1, it will be in row 2.

In [ ]:
combined_rows = []
for idx, item in enumerate(row1):
    # The below condition will look for NaN and empty values
    if item == item:
        combined_rows.append(item)
    else:
        combined_rows.append(row2[idx])

In [ ]:
# Check they are the same length
len(combined_rows) == len(row1)

We can then assign the `columns` property to this new list

In [ ]:
df.columns = combined_rows

In [ ]:
df.head()

Let's now drop those first two rows, as we have already gotten the info from them...

In [ ]:
df = df.drop([0,1])

In [ ]:
df.head()

#### Missing data

Lets look for missing data by looking at count 

In [ ]:
df.describe()

Clearly too much information, lets reduce data to just the important stuff

# 3. Removing unwanted columns
Our data has quite a lot of information in it (265 columns). Let's look at how you would create a subset...


In [ ]:
df.columns

Create a list of `desired_cols` to subset the data by.

In [ ]:
desired_cols = ['HH_ID', 'site.region.name', 'Area of sample parcel  (acres)',\
                'Price of seed planted (MMK)', 'Quantity of crop harvested (kg)',\
                'Quantity of crop sold (kg)', 'Total value of crop sold (MMK)']

In [ ]:
maize_df = df[desired_cols]

In [ ]:
maize_df.head(10)

Ah, now we have duplicates...

In [ ]:
maize_df.columns.duplicated()

Below, I have googled a solution for removing duplicate columns (this is not perfect, but serves as a quick fix).
I found this answer here: https://stackoverflow.com/questions/32041245/fast-method-for-removing-duplicate-columns-in-pandas-dataframe 

In [ ]:
maize_df = maize_df.loc[:,~maize_df.columns.duplicated()]

In [ ]:
maize_df.head()

Looks like some of the columns are misnamed, e.g. price is in dollars and MMK but called the same

Now we've selected some columns of interest, let's simplify some of the column names... 

In [ ]:
simplified_col_names = {'HH_ID':'ID', 'site.region.name':'Region'}

`.rename` will let us rename columns

In [ ]:
maize_df = maize_df.rename(columns=simplified_col_names)

In [ ]:
maize_df.head()

Finally, let's convert all of the numeric data to float types for later calculation 

In [ ]:
# This will show us that our data is all object types 
maize_df.dtypes

In [ ]:
# We know that these are our numeric data types
list(maize_df.columns[2:])

In [ ]:
numeric_cols = list(maize_df.columns[2:])

Pandas has a built-in function that converts data from object to numeric, let's use that... 

In [ ]:
for num_col in numeric_cols:
    maize_df[num_col] = pd.to_numeric(maize_df[num_col])


In [ ]:
# Now the data types have changed
maize_df.dtypes

*Note:* you can also achieve this without a for loop using the .apply() method `maize_df = maize_df[numeric_cols].apply(pd.to_numeric)`

# 4. Imputing values

In [ ]:
# Find columns with at least one NaN value
columns_with_nan = maize_df.columns[maize_df.isnull().any()].tolist()
columns_with_nan

There are several rows with NaN in multiple columns, lets drop them if they are NaN in 4 out of 8 columns

In [ ]:
maize_df = maize_df.dropna(subset=maize_df.columns[maize_df.isnull().sum() >= 4], how='all')

In [ ]:
maize_df

In [ ]:
rows_with_nan = maize_df[maize_df[columns_with_nan[0]].isnull()]
rows_with_nan

In [ ]:
mean_value = maize_df[columns_with_nan[0]].mean()

In [ ]:
mean_value

In [ ]:
maize_df[columns_with_nan[0]].fillna(mean_value, inplace=True)
maize_df[maize_df['ID'] == 65026]

In [ ]:
rows_with_nan = maize_df[maize_df[columns_with_nan[1]].isnull()]
rows_with_nan

In [ ]:
rows_with_nan.index

In the case for quantity harvested lets set it to the quantity sold

In [ ]:
maize_df[columns_with_nan[1]].fillna(maize_df['Quantity of crop sold (kg)'], inplace=True)
maize_df.iloc[rows_with_nan.index]

In [ ]:
rows_with_nan = maize_df[maize_df[columns_with_nan[2]].isnull()]
rows_with_nan

In [ ]:
rows_with_nan = maize_df[maize_df[columns_with_nan[2]].isnull()]

maize_df[columns_with_nan[2]].fillna(maize_df['Quantity of crop harvested (kg)'], inplace=True)
maize_df.iloc[rows_with_nan.index]

This last row is tricky, any sugestions?

In [ ]:
rows_with_nan = maize_df[maize_df[columns_with_nan[3]].isnull()]
rows_with_nan

My idea: find the average price the crops were sold for from other columns and then replace quantity harvested x average priice sold for

In [ ]:
maize_df['Average price sold for'] =  maize_df['Total value of crop sold (MMK)'] / maize_df['Quantity of crop harvested (kg)'] 
maize_df

In [ ]:
mean_price = maize_df['Average price sold for'].mean()
mean_price

In [ ]:
maize_df[columns_with_nan[3]].fillna(maize_df['Quantity of crop sold (kg)']*mean_price, inplace=True)
maize_df.iloc[rows_with_nan.index]

#### 🤨TASK

Now employ an imputing strategy for NaN values in 'Average price sold for'

# 5. Summary statistics + basic plotting


In [ ]:
maize_df.describe()

### Line plots

In [ ]:
maize_df['Quantity of crop harvested (kg)'].plot()

In [ ]:
maize_df['Quantity of crop sold (kg)'].plot()

We can also easily combine plots

In [ ]:
maize_df['Quantity of crop harvested (kg)'].plot()
maize_df['Quantity of crop sold (kg)'].plot()

As the above plots are a little messy, let's take a subset of data.  
In the example below, we take rows where the price of seeds are above a certain limit.

In [ ]:
pricey_maize_df = maize_df.loc[maize_df['Price of seed planted (MMK)'] > 7500]

#### 🤨 TASK
Look at, get some summary statistics for, and plot this new dataframe

## Histograms
Pandas Series also provide a `.hist` method for histograms.

In [ ]:
maize_df['Area of sample parcel  (acres)'].hist()

In [ ]:
# define some smaller bins
bins_to_use = np.arange(0, 50, 1)
bins_to_use

In [ ]:
maize_df['Area of sample parcel  (acres)'].hist(bins=bins_to_use)

### Basics of matplotlib.pyplot (plt)
Matplotlib.pyplot (here loaded in as `plt`), open up some tools for labelling and controlling figures returned by Pandas.
This includes allowing us to set titles, labels for axes, sizing of figure, background colour etc.
For now, I will show you a very basic and well formatted way of using plt

In [ ]:
fig, ax = plt.subplots(1) # This line is something you will grow used to over time. It sets the figure and axes of a plot 
maize_df['Area of sample parcel  (acres)'].hist(bins=bins_to_use, ax=ax) # We pass the argument ax=ax to set the location of the plot

## Set the title, and axes labels.
ax.set_title("Area of Sample Parcel")
ax.set_ylabel("count")
ax.set_xlabel("Area size (acres)")

#### 🤨 Bigger TASK
Let's use what we have learnt to look and plot some derived information from our data.
Let's calculate the total cost of *the difference* between the crop sold and harvested.

*Advice:* make sure you use multiple cells to seperate the tasks and keep the code clean

##### Steps
1. Create a new column `Crop difference (kg)` that is the quantity of crop harvested minus the quantity of crop sold.
2. Convert this column to a difference using `abs()` (google if unsure)
3. Convert the new column from kg to grams and calculate the price (in MMK) of each gram lost (let's assume that `Price of seed planted (MMK)` is for 1g). 
4. Save this a new column e.g. `Total cost of crop difference (MMK)`
5. Plot a histogram and view data

In [ ]:
???

# 6. Locating items / filtering / sampling


In [ ]:
maize_df.describe()

Looks like some of the data is incorrect as there is negative crops lost after harvest, lets investigate this

In [ ]:
## Create a new column
maize_df['Crop lost after harvest (kg)'] = maize_df['Quantity of crop harvested (kg)']  - maize_df['Quantity of crop sold (kg)'] 

In [ ]:
## let's look at values less than 0 kg
maize_df[maize_df['Crop lost after harvest (kg)'] < 0]

This is clearly incorrect and as we imputed values with the value in the other column the difference should be 0. For these cases we can do one of two things: 
1. drop the rows
2. replace the difference with 0 

What seems more appropriate?

#### 🤨TASK

employ your prefered strategy

Lets look at the best performing farms, i.e. the farms which had the highest total value sold and the highest percentage crops harvested

In [ ]:
bins_to_use = np.arange(0, 100000, 1000)
bins_to_use
maize_df['Quantity of crop harvested (kg)'].hist(bins=bins_to_use)

In [ ]:
maize_df[maize_df['Quantity of crop harvested (kg)']>10000]

#### 🤨TASK
*Advice:* make sure you use multiple cells to seperate the tasks and keep the code clean

Make new column with percentage of crops lost 

#### 🤨TASK

Find the well performing farms with a low percentage (10%) of crops lost

In [ ]:

???

#### Sampling
For many statistical applications you may need to use sampling when you have uneven categories of things. Pandas provides a very simple solution to sampling with the `.sample()` method...

In [ ]:
maize_df.sample(10)

# 7. Grouping / pivoting + plotting
Finally, we will briefly cover more advanced pandas methods for handling and grouping data. 
Firstly `.groupby()` which will allow us to group categorical variables i.e. such as Region

In [ ]:
# Let's look at the regional mean values
maize_df.groupby('Region').mean()

#### 🤨TASK
In a new cell below, save a new variable `regional_means` which is a groupby of the Regional means and then make a barplot of "Price of seed planted (MMK)". Which region planted the most expensive seeds on average?

Okay, so we've looked at the regional mean, what if we wanted to get a sum of a column by region? 

In [ ]:
# Take a sum of only the total crop sold values (we can use the .agg function to only get this value)
crops_sold_by_region = maize_df.groupby('Region').agg({'Total value of crop sold (MMK)':'sum'})

Note: if you wanted the sum of one column and the mean of another the syntax would be like:  
`df.groupby('col').agg({'col1':'sum', 'col2':'mean'})`

Let's plot this data

In [ ]:
crops_sold_by_region.head()

In [ ]:
crops_sold_by_region.plot()

Let's use matplotlib.pyplot (plt) to clean up and make this figure a little nicer...

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 3))
crops_sold_by_region.plot(marker='s', ax=ax)

#### 🤨TASK
In the cell above, add a title, y and x label.

*Remember:* Google may help

**Further Tasks** Set the font size of the title and axes labels. 
Also look at changing the markers to circles and the color of line to black...


### Pivoting
Another form of data handling that pandas opens up is pivoting with `.pivot()`.

Firstly, let's get something worthwhile pivoting the data on. Let's look at the influence of groups of area size to groups of harvest amount i.e. can we prove with this data that larger plot sizes mean more harvest in kg?

Let's take the 33rd and 66th percentile as cut off points for our groups. We use pandas inbuilt function `.quantile` to get these values...

In [ ]:
## Let's look at the 33rd and 66th percentile values for area sizes
maize_df['Area of sample parcel  (acres)'].quantile([0.33, 0.66])

Next, let's define a simple function that we can `.apply` to our dataset to label the data based on these quantile values of 1.5 and 3.0

In [ ]:
def label_data_based_on_area(row):
    # Simple function to label data in Area of sample parcel column
    if row['Area of sample parcel  (acres)'] > 3:
        return "Area > 3 acres"
    elif row['Area of sample parcel  (acres)'] > 1.5 and row['Area of sample parcel  (acres)'] < 3:
        return "Area between 1.5 and 3 acres"
    else:
        return "Area < 1.5 acres"


In [ ]:
# Create new data column
maize_df['Area size group'] = maize_df.apply(label_data_based_on_area, axis=1)

In [ ]:
maize_df['Area size group'] 

These labels look good, let's now do the same for the quantity of crop harvested...

In [ ]:
## Let's look at the 33rd and 66th percentile values of crop harvested sizes
maize_df['Quantity of crop harvested (kg)'].quantile([0.33, 0.66])

And define another very similar function.
For more efficient code, you want to avoid very similar functions (i.e. functions that share a large % of code), but for this example it is okay...

In [ ]:
def label_data_based_on_crop_harvested(row):
    # Another simple function to label data.
    if row['Quantity of crop harvested (kg)'] > 3260:
        return "Harvest > 3260 kg"
    elif row['Quantity of crop harvested (kg)'] > 1304 and row['Quantity of crop harvested (kg)'] < 3260:
        return "Harvest between 1304 and 3260 kg"
    else:
        return "Harvest < 1304 kg"


In [ ]:
# Apply this function and create a new column
maize_df['Harvest size group'] = maize_df.apply(label_data_based_on_crop_harvested, axis=1)

In [ ]:
maize_df['Harvest size group'] 

Now, let's pivot based on these two new columns and see what we get 

In [ ]:
maize_df_harvest_area_pivot = maize_df.pivot(index='ID', columns='Harvest size group', values='Area size group')

In [ ]:
# This is a breakdown of Harvest size to Area size
maize_df_harvest_area_pivot

Let's count the values of each column with `.value_counts()`

In [ ]:
maize_df_harvest_area_pivot['Harvest < 1304 kg'].value_counts()

In [ ]:
maize_df_harvest_area_pivot['Harvest > 3260 kg'].value_counts()

#### 🤨TASK
Have a play around and determine whether you could say that one size of area is more efficient for quantity of harvest than the others based only on the labels we have created here. 

*Hint:* Perhaps use .sample to create equally sized subsets, as these three categories have different amounts of values

**FURTHER TASK** Create a visualisation to show this.

In [ ]:
???

## Thanks!